Error calling Gemini API: HTTPSConnectionPool(host='api.generativeai.google.com', port=443): Max retries exceeded with url: /v1beta2/models/gemini-pro:generateText (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020969D957F0>: Failed to resolve 'api.generativeai.google.com' ([Errno 11001] getaddrinfo failed)"))
Error calling Gemini API: HTTPSConnectionPool(host='api.generativeai.google.com', port=443): Max retries exceeded with url: /v1beta2/models/gemini-pro:generateText (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020969CD0E10>: Failed to resolve 'api.generativeai.google.com' ([Errno 11001] getaddrinfo failed)"))
Error calling Gemini API: HTTPSConnectionPool(host='api.generativeai.google.com', port=443): Max retries exceeded with url: /v1beta2/models/gemini-pro:generateText (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000020969CD3D90>: Failed to resolve 'api.generativeai.goo

Phản hồi từ Gemini: Hi there! How can I help you?


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import UnexpectedAlertPresentException,ElementNotInteractableException, WebDriverException,NoSuchElementException, TimeoutException, StaleElementReferenceException
import random
from selenium.webdriver.common.action_chains import ActionChains
import string
import re

In [2]:
# Set up the WebDriver
driver = webdriver.Chrome()

driver.get("http://localhost:9999/DEMO/login")

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


In [3]:
##check login
def test_login_success():
    driver.get("http://localhost:9999/DEMO/home")
    time.sleep(1.5)
    driver.find_element(By.ID, "username").send_keys("admin")
    time.sleep(1.5)
    driver.find_element(By.ID, "password").send_keys("123")
    time.sleep(0.5)
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    
    # Wait for the redirection
    WebDriverWait(driver, 10).until(EC.url_to_be("http://localhost:9999/DEMO/home"))

    assert driver.current_url == "http://localhost:9999/DEMO/home"
def test_login_failed_invalid_username():
    driver.get("http://localhost:9999/DEMO/login")
    time.sleep(2)
    driver.find_element(By.ID, "username").send_keys("ngu")
    time.sleep(2)
    driver.find_element(By.ID, "password").send_keys("123")
    time.sleep(1)
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    
    # Wait for the error message to appear
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "error-message")))
    
    # Locate the error message and verify its content
    error_message = driver.find_element(By.CLASS_NAME, "error-message").text
    assert error_message == "Wrong username or password"

In [4]:
test_login_success()

In [25]:
test_login_failed_invalid_username()

In [5]:
def test_add_to_cart(driver, url, x):
    """Adds a product to the cart with a random quantity (including floats) and asserts the results on the order page."""
    try:
        driver.get(url)

        try:
            add_to_cart_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "(//a[contains(@class, 'btn') and contains(., 'Add to Cart')])[{}]".format(x)))
            )
            add_to_cart_button.click()
        except TimeoutException:
            print(f"Error: Could not find the 'Add to Cart' button for product index {x}. Please check the product count.")
            return
        if x>7:
            quantity_input = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, f"quantity_{x+999}"))
        )
        # Wait for the quantity input field to appear
        else:
            quantity_input = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, f"quantity_{x}"))
            )

        try:
            # Locate the max quantity element
            max_quantity_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//p[@class='quantity-info']"))
            )
            max_quantity_text = max_quantity_element.text.strip()
            # Extract the numeric value as a float
            max_quantity = float(max_quantity_text.split()[1])
            if max_quantity <= 0:
                max_quantity = 1.0
        except (NoSuchElementException, ValueError) as e:
            print(f"Error: Could not extract max quantity: {e}")
            max_quantity = 1.0

        # Generate a random quantity (float) within the allowed range
        quantity = round(random.uniform(1.0, max_quantity), 1)

        try:
            # Input the random quantity
            quantity_input.clear()
            quantity_input.send_keys(str(quantity))

            # Locate and click the submit button
            submit_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))
            )
            submit_button.click()
            time.sleep(2)  # Crucial delay for processing
        except (NoSuchElementException, StaleElementReferenceException, WebDriverException, ElementNotInteractableException) as e:
            print(f"Error during quantity input or submit: {e}")
            raise
        except TimeoutException as e:
            print(f"Timed out waiting for elements. Check page and locators. Error: {e}")
            raise

        # Wait for the redirection to the order page
        WebDriverWait(driver, 20).until(EC.url_to_be("http://localhost:9999/DEMO/view/order/order.jsp"))
        print("Successfully redirected to order.jsp")

        # Additional assertions for the order page can be added here as needed.

    except TimeoutException as e:
        print(f"Timeout error during any step: {e}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise


In [6]:
url = "http://localhost:9999/DEMO/home"
test_add_to_cart(driver, url, 1)

Successfully redirected to order.jsp


In [7]:
##create an random address
def generate_random_text(min_length=5, max_length=20):
    length = random.randint(min_length, max_length)
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

##test click submit order
def test_place_order(driver, url):
    try:
        driver.get(url)

        # ... (Previous code to add to cart, input quantity, etc.) ...

        try:
            address_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "address"))
            )
            address_text = generate_random_text()
            address_input.send_keys(address_text)

            # Crucial: Find the button with the correct locator.  The provided XPATH is likely incorrect
            place_order_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@type='submit' and @class='submit-btn']"))
                #or  EC.element_to_be_clickable((By.XPATH, "//button[text()='Place Order']")) if the text is exact
            )  # Use a more precise locator

            if place_order_button.is_enabled():
                place_order_button.click()
                time.sleep(2) #Crucially a short wait
            else:
              print("Place order button is not enabled")
              raise Exception("Button not enabled")
        
        except (NoSuchElementException, StaleElementReferenceException, WebDriverException, ElementNotInteractableException, TimeoutException) as e:
            print(f"Error during address input, button click, or order placement: {e}")
            raise

            # 1.  Wait for the alert to appear and check it
        try:
             alert = WebDriverWait(driver, 10).until(EC.alert_is_present())
             alert_text = alert.text
             assert alert_text == "Order placed successfully!", f"Alert message is not correct. Actual: {alert_text}"
             alert.accept() #Dismiss the alert
        except TimeoutException:
            raise TimeoutException ("Alert not present in the specified timeout")
        except UnexpectedAlertPresentException as e:
           print(f"An unexpected alert was present: {e}")
           raise

    except TimeoutException as e:
        print(f"Timeout error during any step: {e}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

In [8]:
url="http://localhost:9999/DEMO/view/order/order.jsp"
test_place_order(driver,url)

In [40]:
##wrong input in add to cart
def test_add_to_cart_wrong(driver, url, x):
    """Adds a product to the cart with a random quantity."""
    try:
        driver.get(url)

        # Find and click the "Add to Cart" button.  Crucially, use presence_of_element_located
        add_to_cart_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "(//a[contains(@class, 'btn') and contains(., 'Add to Cart')])[{}]".format(x)))
        )
        add_to_cart_button.click()

        # Wait for the quantity input field to appear
        if x>7:
            quantity_input = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, f"quantity_{x+999}"))
            )
        # Wait for the quantity input field to appear
        else:
            quantity_input = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, f"quantity_{x}"))
            )

        try:
            max_quantity_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//p[@class='quantity-info']"))
            )
            max_quantity_text = max_quantity_element.text.strip()
            max_quantity = int(max_quantity_text.split()[1])
            if max_quantity <= 0:
                max_quantity = 1
        except (NoSuchElementException, ValueError) as e:
            print(f"Error: Could not extract max quantity: {e}")
            max_quantity = 1

        invalid_quantity = random.choice(
            [random.randint(-100, 0), random.randint(max_quantity + 1, max_quantity + 100)]
        )

        try:
            quantity_input.clear()
            quantity_input.send_keys(str(invalid_quantity))


            # **Crucial:**  More robust locator strategies for the submit button
            submit_button = WebDriverWait(driver, 20).until(
                #Try CSS Selector (best effort):
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))
            )

            # If CSS selector doesn't work, try more specific XPATH
            if submit_button is None:
              submit_button = WebDriverWait(driver, 20).until(
                  EC.element_to_be_clickable((By.XPATH, "//button[contains(@type, 'submit') and contains(., 'Confirm')]"))  
              )


            if submit_button is not None:
               submit_button.click()
               time.sleep(2) # Crucial delay
            else:
                print("Could not find submit button with either locator.")
                raise Exception("Submit button not found")

        except (NoSuchElementException, StaleElementReferenceException, WebDriverException, ElementNotInteractableException) as e:
            print(f"Error during quantity input or submit: {e}")
            raise
        except TimeoutException as e:
          print(f"Timed out waiting for elements. Check page and locators. Error: {e}")
          raise
          

    except TimeoutException as e:
        print(f"Timeout error during any step: {e}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

In [44]:
url = "http://localhost:9999/DEMO/home"  # Replace with the actual URL
x=3
test_add_to_cart_wrong(driver, url, x)

Error: Could not extract max quantity: invalid literal for int() with base 10: '100.0'


In [ ]:
#           Methods that have the same function



# 1. Find 